<a href="https://colab.research.google.com/github/yukismd/DL_for_ImageData_and_Finetuning/blob/main/text_classification_hatespeech/PT/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!ls

sample_data


In [5]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [6]:
import datasets
print(datasets.__version__)
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
import evaluate
print(evaluate.__version__)

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tqdm.auto import tqdm

2.19.1
4.41.0
0.4.2


In [27]:
df = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_train.csv')
print(df.shape)
df.head()

(4656, 5)


,id,source,text,label,hold
0,b1b0d5e6c,newsplus,手引きしたのは高い確率で大院君だよなぁ,0,2
1,a5e29c5f1,livejupiter,いや別にワイが困るわけじゃないからええけど対応大変やなぁと思ってな,0,1
2,bd72cb57e,newsplus,そこで家族会すら総スカンを食らった極左を出すあたり頭が逝ってるな\n結局動かぬ証拠だから動か...,0,4
3,66387bae8,news4vip,もともとB'zが好きだったんだが松本のソロアルバムでtake5カバーしててな\nそれで原曲聴...,0,1
4,b80dc3a94,livejupiter,スタンドって本体の内面現すんだよな\n\nいろいろ想像するとおもろいわ\n何でDIOは時止め...,0,2


In [30]:
df_train = df[df['hold'] != 0]
df_valid = df[df['hold'] == 0]
df_train.shape, df_valid.shape

((3759, 5), (897, 5))

In [32]:
ds_train = datasets.Dataset.from_pandas(df_train[['text','label']], preserve_index=False)
ds_valid = datasets.Dataset.from_pandas(df_valid[['text','label']], preserve_index=False)

In [33]:
#ds_train = datasets.load_dataset('csv', data_files=['https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_train.csv'])
ds_train

Dataset({
    features: ['text', 'label'],
    num_rows: 3759
})

In [36]:
ds_valid

Dataset({
    features: ['text', 'label'],
    num_rows: 897
})

In [25]:
ds_train['train']['hold']

[2,
 1,
 4,
 1,
 2,
 1,
 1,
 3,
 1,
 3,
 0,
 2,
 1,
 3,
 2,
 1,
 2,
 3,
 0,
 1,
 3,
 4,
 0,
 3,
 4,
 4,
 3,
 3,
 1,
 3,
 2,
 3,
 0,
 2,
 4,
 3,
 3,
 4,
 3,
 0,
 3,
 0,
 0,
 3,
 2,
 4,
 4,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 3,
 0,
 2,
 4,
 1,
 2,
 3,
 3,
 0,
 3,
 0,
 2,
 1,
 3,
 4,
 4,
 0,
 3,
 1,
 0,
 3,
 0,
 2,
 1,
 4,
 1,
 2,
 4,
 0,
 3,
 2,
 4,
 1,
 4,
 1,
 1,
 3,
 0,
 0,
 0,
 2,
 2,
 0,
 4,
 2,
 0,
 3,
 2,
 4,
 4,
 2,
 3,
 0,
 4,
 1,
 1,
 4,
 1,
 2,
 3,
 0,
 0,
 1,
 3,
 3,
 0,
 1,
 3,
 2,
 0,
 4,
 1,
 3,
 4,
 4,
 1,
 0,
 1,
 0,
 1,
 4,
 3,
 2,
 0,
 0,
 4,
 2,
 1,
 1,
 0,
 3,
 2,
 0,
 2,
 4,
 2,
 4,
 1,
 0,
 0,
 2,
 2,
 2,
 3,
 1,
 1,
 2,
 3,
 4,
 2,
 3,
 1,
 4,
 0,
 3,
 3,
 1,
 4,
 1,
 4,
 2,
 2,
 0,
 0,
 4,
 1,
 4,
 0,
 0,
 4,
 0,
 3,
 1,
 3,
 3,
 2,
 0,
 0,
 4,
 0,
 3,
 1,
 1,
 1,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 4,
 0,
 4,
 2,
 4,
 2,
 1,
 3,
 1,
 0,
 0,
 0,
 0,
 1,
 2,
 3,
 2,
 1,
 1,
 3,
 4,
 2,
 3,
 4,
 3,
 3,
 2,
 2,
 1,
 3,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 1,
 4,
